In [ ]:
!pip install torch transformers

In [ ]:
import torch
import time
import os
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from torch.utils.data import Dataset, DataLoader
from google.colab import drive

# Define a simple dataset for training
class SimpleDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length=128):
        tokenizer.pad_token = tokenizer.eos_token  # Set padding token to EOS
        self.encodings = tokenizer(texts, padding=True, truncation=True, max_length=max_length)

    def __len__(self):
        return len(self.encodings["input_ids"])

    def __getitem__(self, idx):
        return {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}

drive.mount('/content/drive')

# Load pretrained GPT-2 model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('/content/drive/My Drive/GPT-2/Amulet_1.0/outputTest_checkpoint')
model = GPT2LMHeadModel.from_pretrained('/content/drive/My Drive/GPT-2/Amulet_1.0/outputTest_checkpoint')

# Check if GPU is available and set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  # Move model to GPU if available


qa_data = []

# Read the file
with open("/content/drive/My Drive/GPT-2/Amulet_1.0/outputTest.txt", "r") as file:
    lines = file.read().splitlines()

    # Loop through the lines to store data
    for line in lines:
        qa_data.append((line))

dataset = SimpleDataset(qa_data, tokenizer)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

# Set up the optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

# Set model to training mode
model.train()

# Training loop (mock training)
i = 0
c = time.time_ns()/(60*1e9)
for batch in dataloader:
    # Move input tensors to GPU if available
    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)

    # Zero the gradients
    optimizer.zero_grad()

    # Forward pass
    outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
    loss = outputs.loss

    # Backward pass
    loss.backward()

    # Update weights
    optimizer.step()

    if(i % 1000 == 0):
        i = 0
        print(f"Loss: {loss.item()} Time Elapsed: {(time.time_ns()/(60*1e9) - c):.9f}")  # Print loss for monitoring
        model.save_pretrained('/content/drive/My Drive/GPT-2/Amulet_1.0/outputTest_checkpoint')
        tokenizer.save_pretrained('/content/drive/My Drive/GPT-2/Amulet_1.0/outputTest_checkpoint')
        c = time.time_ns()/(60*1e9)
    i += 1

# Save the final model and tokenizer
model.save_pretrained('/content/drive/My Drive/GPT-2/Amulet_1.0')
tokenizer.save_pretrained('/content/drive/My Drive/GPT-2/Amulet_1.0')


In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from google.colab import drive

# Load the trained model and tokenizer
#model = GPT2LMHeadModel.from_pretrained("gpt2")
#tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
#model.eval()

# Function to ask a question and get an answer
def ask_question(question):
    input_text = f"Question: {question} Answer:"
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    attention_mask = torch.ones(input_ids.shape, dtype=torch.long)

    with torch.no_grad():
        output = model.generate(
            input_ids,
            attention_mask=attention_mask,
            max_length=50,
            num_return_sequences=1,
            no_repeat_ngram_size=2,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            temperature=0.7
        )

    answer = tokenizer.decode(output[0], skip_special_tokens=True)
    return answer.split("Answer:")[-1].strip()  # Return the generated answer

# Example questions
#print(ask_question("Where am I?"))

drive.mount('/content/drive')


model = GPT2LMHeadModel.from_pretrained("/content/drive/My Drive/GPT-2/Amulet_1.0/outputTest_checkpoint")
tokenizer = GPT2Tokenizer.from_pretrained("/content/drive/My Drive/GPT-2/Amulet_1.0/outputTest_checkpoint")
model.eval()


print(ask_question("Who has the Bing Chilling?"))

